<center>
## Открытый курс по машинному обучению. Сессия №3
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

## <center> Домашнее задание № 1
## <center> Анализ данных по сердечно-сосудистым заболеваниям

В соревновании предлагалось определить наличие/отсутствие сердечно-сосудистых заболеваний (ССЗ) по результатам осмотра пациента.

**Описание данных.**

Датасет сформирован из реальных клинических анализов, и в нём используются признаки, которые можно разбить на 3 группы:

Объективные признаки:

 - Возраст (age)
 - Рост (height)
 - Вес (weight)
 - Пол (gender)
 

Результаты измерения:

 - Артериальное давление верхнее и нижнее (ap_hi, ap_lo)
 - Холестерин (cholesterol)
 - Глюкоза (gluc)
 

Субъективные признаки (со слов пациентов):

 - Курение (smoke)
 - Употребление алкоголя (alco)
 - Физическая активность (active)
 
Целевой признак (который интересно будет прогнозировать):
 - Наличие сердечно-сосудистых заболеваний по результатам классического врачебного осмотра (cardio)

Значения показателей холестерина и глюкозы представлены одним из трех классов: норма, выше нормы, значительно выше нормы. Значения субъективных признаков — бинарны.

Все показатели даны на момент осмотра.

Мы будем работать только с обучающей выборкой и с помощью `Pandas` проведем первичный анализ данных. 

Из библиотек нам понадобятся только `NumPy` и `Pandas`.

In [2]:
import numpy as np
import pandas as pd

Считываем данные из CSV-файла в объект pandas DataFrame.  

In [3]:
df = pd.read_csv('mlbootcamp5_train.csv', sep=';', 
                 index_col='id')

Посмотрим на первые 5 записей.

In [4]:
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 0 to 99999
Data columns (total 12 columns):
age            70000 non-null int64
gender         70000 non-null int64
height         70000 non-null int64
weight         70000 non-null float64
ap_hi          70000 non-null int64
ap_lo          70000 non-null int64
cholesterol    70000 non-null int64
gluc           70000 non-null int64
smoke          70000 non-null int64
alco           70000 non-null int64
active         70000 non-null int64
cardio         70000 non-null int64
dtypes: float64(1), int64(11)
memory usage: 6.9 MB
None


**<font color='red'>Вопрос 1 (1 балл).</font> Сколько мужчин и женщин представлено в этом наборе данных? Не было дано расшифровки признака "пол" (какому полу соответствует 1, а какому – 2 в признаке ``gender``) – это определите, посмотрев также на рост при разумном предположении, что в среднем мужчины выше (здесь и далее под средним понимается среднее арифметическое).**

**<font color='red'>Варианты:</font>**
- 45470 женщин и 24530 мужчин
- 45470 мужчин и 24530 женщин
- 45530 женщин и 24470 мужчин
- 45530 мужчин и 24470 женщин

In [6]:
# Средний рост женщин
print(int(df[df['gender'] == 1]['height'].mean()), 'см при gender=1.')

# Средний рост мужчин
print(int(df[df['gender'] == 2]['height'].mean()), 'см при gender=2.')

# Число женщин и мужчин соотвественно
print(df[df['gender'] == 1]['gender'].count(), 'женщин')
print(df[df['gender'] == 2]['gender'].count(), 'мужчин')

161 см при gender=1.
169 см при gender=2.
45530 женщин
24470 мужчин


**<font color='red'>Вопрос 2 (1 балл).</font> Кто в среднем реже указывает, что употребляет алкоголь – мужчины или женщины?**

**<font color='red'>Варианты:</font>**
- мужчины 
- женщины

In [7]:
# Доли употребляющих алкоголь
drinking_women = df[(df['gender'] == 1) & (df['alco'] == 1)]['gender'].count() / df[df['gender'] == 1]['gender'].count()
drinking_men = df[(df['gender'] == 2) & (df['alco'] == 1)]['gender'].count() / df[df['gender'] == 2]['gender'].count()

# + Число употребляющих алкоголь
print("Женщины: {} | {}".format(df[(df['gender'] == 1) & (df['alco'] == 1)]['gender'].count(), drinking_women.round(3)))
print("Мужчины: {} | {}".format(df[(df['gender'] == 2) & (df['alco'] == 1)]['gender'].count(), drinking_men.round(3)))

Женщины: 1161 | 0.025
Мужчины: 2603 | 0.106


**<font color='red'>Вопрос 3 (1 балл).</font> Во сколько раз (округленно, `round`) процент курящих среди мужчин больше, чем процент курящих среди женщин (по крайней мере, по этим анкетным данным)?**

**<font color='red'>Варианты:</font>**
- 4
- 8
- 12
- 16

In [8]:
# Процент курящих женщин и мужчин соотвественно
women = ((df[(df['gender'] == 1) & (df['smoke'] == 1)]['gender'].count() /
          df[df['gender'] == 1]['gender'].count()) * 100).round(2)
print('Процент курящих женщин: {}.'.format(women))

men = ((df[(df['gender'] == 2) & (df['smoke'] == 1)]['gender'].count() /
        df[df['gender'] == 2]['gender'].count()) * 100).round(2)
print('Процент курящих мужчин: {}.'.format(men))

# Отличие показателей
diff = (men / women).round()
print('Примерно в', int(diff), 'раз.')

Процент курящих женщин: 1.79.
Процент курящих мужчин: 21.89.
Примерно в 12 раз.


**<font color='red'>Вопрос 4 (1 балл).</font> Вы наверняка заметили, что значения возраста какие-то странные. Догадайтесь, в чём здесь измеряется возраст, и ответьте, на сколько месяцев (примерно) отличаются медианные значения возраста курящих и некурящих.**

**<font color='red'>Варианты:</font>**
- 10
- 15
- 20
- 25

In [10]:
smokers = int(df[df['smoke'] == 1]['age'].median() // 30)
print("Средний возраст курящих:", smokers)

non_smokers = int(df[df['smoke'] == 0]['age'].median() // 30)
print("Средний возраст некурящих:", non_smokers)

# Отличие медианных значений
diff = max(smokers, non_smokers) - min(smokers, non_smokers)

print("Примерно на {} месяцев.".format(diff))

Средний возраст курящих: 637
Средний возраст некурящих: 657
Примерно на 20 месяцев.


**<font color='red'>Вопрос 5 (2 балла).</font> В [статье](https://ru.wikipedia.org/wiki/Сердечно-сосудистый_риск) на Википедии про сердечно-сосудистый риск показана шкала SCORE для расчёта риска смерти от сердечно-сосудистого заболевания в ближайшие 10 лет. Вот она:**
<img src='https://habrastorage.org/webt/gg/38/vf/gg38vfl5ql8obqjmlsvt9es76p8.png' width=70%>

Давайте посмотрим на правый верхний прямоугольник, отображающий сегмент курящих мужчин в возрасте от 60 до 64 лет включительно. (Неочевидно, но тут для возраста и давления цифры означают верхнюю границу, и она не включается).

Видим 9-ку в левом нижнем углу этого прямоугольника и 47 – в правом верхнем. То есть если при этом систолическое (т.е. верхнее) артериальное давление – меньше 120 мм рт.ст., а уровень холестерина – 4 ммоль/л, то риск ССЗ оценивается примерно в 5 раз ниже, чем если бы давление лежало в интервале [160, 180), а холестерина было бы 8 ммоль/л.

Давайте посчитаем аналогичное значение, но на наших данных. 

Уточнения:
- Создайте новый признак ``age_years`` – возраст в годах, округлив до целых (`round`). Для данного примера отберите курящих мужчин от 60 до 64 лет включительно
- Категории уровня холестрина на рисунке и в наших данных отличаются. Отображение значений на картинке в значения признака ``cholesterol`` следующее: 4 ммоль/л $\rightarrow$ 1, 5-7 ммоль/л $\rightarrow$ 2, 8 ммоль/л $\rightarrow$ 3.
- Интересуют 2 подвыборки курящих мужчин возраста от 60 до 64 лет включительно: первая с верхним артериальным давлением строго меньше 120 мм рт.ст. и концентрацией холестерина – 4 ммоль/л, а вторая – с верхним артериальным давлением от 160 (включительно) до 180 мм рт.ст. (не включительно) и концентрацией холестерина – 8 ммоль/л.

Во сколько раз (округленно, `round`) отличаются доли больных людей (согласно целевому признаку, `cardio`) в этих двух подвыборках? Посчитайте на наших данных.

**<font color='red'>Варианты:</font>**
- 2
- 3
- 4
- 5

In [11]:
# Данные о курящих мужчинах 60-64 лет
df['age_years'] = (df['age'] / 365).round()
data = df[(df['gender'] == 2) & (df['smoke'] == 1) & (60 <= df['age_years']) & (df['age_years'] <= 64)]

# Доля в первой выборке
men_120_ill = data[(data['ap_hi'] < 120) & (data['cholesterol'] == 1) & data['cardio'] == 1]['gender'].count()
men_120 = data[(data['ap_hi'] < 120) & (data['cholesterol'] == 1)]['gender'].count()
print('Доля больных в первой выборке:', (men_120_ill / men_120).round(3))

# Доля во второй выборке
men_180_ill = data[(160 <= data['ap_hi']) & (data['ap_hi'] < 180) & 
                   (data['cholesterol'] == 3) & data['cardio'] == 1]['gender'].count()
men_180 = data[(160 <= data['ap_hi']) & (data['ap_hi'] < 180) & (data['cholesterol'] == 3)]['gender'].count()
print('Доля больных во второй выборке:', (men_180_ill / men_180).round(3))

# Отличие
diff = ((men_180_ill / men_180) / (men_120_ill / men_120)).round()
print('Отличие: примерно в', int(diff), 'раза.')

Доля больных в первой выборке: 0.256
Доля больных во второй выборке: 0.864
Отличие: примерно в 3 раза.


**<font color='red'>Вопрос 6 (2 балла).</font> Постройте новый признак – BMI ([Body Mass Index](https://en.wikipedia.org/wiki/Body_mass_index)). Для этого надо вес в килограммах поделить на квадрат роста в метрах. Нормальными считаются значения BMI от 18.5 до 25. Выберите верные утверждения.**

**<font color='red'>Утверждения:</font>**
- Медианный BMI по выборке превышает норму
- У женщин в среднем BMI ниже, чем у мужчин
- У здоровых в среднем BMI выше, чем у больных
- В сегменте здоровых и непьющих мужчин в среднем BMI ближе к норме, чем в сегменте здоровых и непьющих женщин

In [12]:
# Body Mass Index - Индекс массы тела
df['BMI'] = df['weight'] / ((df['height'] / 100) ** 2)
#df.head(10)

print('1) Медианный BMI по выборке:', df['BMI'].median(), '(при норме от 18.5 до 25).')

# BMI женщин и мужчин соответственно
print('2) BMI женщин:', df[df['gender'] == 1]['BMI'].mean().round(3))
print('   BMI мужчин:', df[df['gender'] == 2]['BMI'].mean().round(3))

# BMI здоровых и больных
print('3) BMI здоровых:', df[df['cardio'] == 0]['BMI'].mean().round(3))
print('   BMI больных:', df[df['cardio'] == 1]['BMI'].mean().round(3))

# Сегменты здоровых и непьющих мужчин и женщин
m = df[(df['gender'] == 2) & (df['cardio'] == 0) & (df['alco'] == 0)]['BMI'].mean()
w = df[(df['gender'] == 1) & (df['cardio'] == 0) & (df['alco'] == 0)]['BMI'].mean()
print('4) BMI здоровых и непьющих мужчин:', m, '| Выше нормы на:', m - 25)
print('   BMI здоровых и непьющих женщин:', w, '| Выше нормы на:', w - 25)

1) Медианный BMI по выборке: 26.374068120774975 (при норме от 18.5 до 25).
2) BMI женщин: 27.988
   BMI мужчин: 26.754
3) BMI здоровых: 26.548
   BMI больных: 28.566
4) BMI здоровых и непьющих мужчин: 25.87263807546017 | Выше нормы на: 0.8726380754601699
   BMI здоровых и непьющих женщин: 26.845406594131507 | Выше нормы на: 1.8454065941315072


**<font color='red'>Вопрос 7 (2 балла).</font> ** Можно заметить, что данные не особо-то чистые, много в них всякой "грязи" и неточностей. Еще лучше мы это увидим, когда обсудим визуализацию данных.

Отфильтруйте следующие сегменты пациентов (считаем это ошибками в данных)
- указанное нижнее значение артериального давления строго выше верхнего
- рост строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили (используйте `pd.Series.quantile`, если не знаете, что это такое – прочитайте)
- вес строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили

Этот вовсе не вся чистка данных, которую можно было проделать, но пока остановимся на этом.

Сколько процентов данных (округленно, `round`) мы выбросили?

**<font color='red'>Варианты:</font>**
- 8
- 9
- 10
- 11

In [15]:
# Указанное нижнее значение артериального давления строго выше верхнего
d1 = df[df['ap_hi'] < df['ap_lo']]

# Рост строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили
d2 = df[(df['height'] < df['height'].quantile(0.025)) | (df['height'] > df['height'].quantile(0.975))]

# Вес строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили
d3 = df[(df['weight'] < df['weight'].quantile(0.025)) | (df['weight'] > df['weight'].quantile(0.975))]

div = (((d1['height'].count() + d2['height'].count() + d3['height'].count()) / df['height'].count()) * 100).round()
print('Выброшено примерно', int(div), 'процентов данных.')

Выброшено примерно 10 процентов данных.
